In [1]:
import pandas as pd
import mysql.connector
import sqlalchemy

In [9]:
cnx = mysql.connector.connect(user='root', password='password', 
                              host='127.0.0.1', database='otus_db', port='3306')

In [10]:
cur = cnx.cursor()

### В этом запросе мы выбираем имена, фамилии, email 10 клиентов,подсчитываем общее количество посещений и покупок для каждого клиента, сумму потраченных ими денег. Запрос также включает подзапрос, который выбирает клиентов имеющих больше 2 посещений.

In [99]:
query = f'''
SELECT
  ci.first_name,
  ci.last_name,
  ci.email,
  COUNT(DISTINCT wv.id) AS total_visits,
  COUNT(DISTINCT p.id) AS total_purchases,
  SUM(p.price) AS total_spent
FROM
  contact_info AS ci
  LEFT JOIN website_visits AS wv ON ci.id = wv.client_id
  LEFT JOIN purchases AS p ON ci.id = p.client_id
WHERE
  ci.id IN (
      SELECT client_id as cnt FROM website_visits group by client_id having count(*) > 2
    ) 
GROUP BY
  ci.id
  having total_spent > 0
ORDER BY
  total_spent DESC
  limit 10;

        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name,last_name,email,total_visits,total_purchases,total_spent
0,Иван,Сидоров,user350@example.com,11,5,44680.02
1,Алексей,Сидоров,user269@example.com,15,4,41746.80
2,Алексей,Петров,user35@example.com,10,6,40880.60
3,Алексей,Петров,user230@example.com,7,7,36403.22
4,Иван,Петров,user883@example.com,10,6,35700.90
5,Иван,Петров,user906@example.com,10,7,35154.40
6,Иван,Сидоров,user673@example.com,11,4,32263.33
7,Алексей,Петров,user839@example.com,10,8,32035.00
8,Алексей,Сидоров,user87@example.com,7,6,29563.17
9,Алексей,Сидоров,user13@example.com,9,5,29231.55


### Команда EXPLAIN

#### Выражение EXPLAIN предоставляет информацию о том, как MySQL выполняет запрос. Оно работает с выражениями SELECT, UPDATE, INSERT, DELETE и REPLACE.

In [82]:
query = f'''
EXPLAIN
SELECT
  ci.first_name,
  ci.last_name,
  ci.email,
  COUNT(DISTINCT wv.id) AS total_visits,
  COUNT(DISTINCT p.id) AS total_purchases,
  SUM(p.price) AS total_spent
FROM
  contact_info AS ci
  LEFT JOIN website_visits AS wv ON ci.id = wv.client_id
  LEFT JOIN purchases AS p ON ci.id = p.client_id
WHERE
  ci.id IN (
      SELECT client_id as cnt FROM website_visits group by client_id having count(*) > 2
    ) 
GROUP BY
  ci.id
  having total_spent > 0
ORDER BY
  total_spent DESC
  limit 10;

        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,ci,None,index,PRIMARY,PRIMARY,4,None,3000,100.0,Using where; Using temporary; Using filesort
1,1,PRIMARY,wv,None,ref,client_id,client_id,5,otus_db.ci.id,1,100.0,Using index
2,1,PRIMARY,p,None,ref,client_id,client_id,5,otus_db.ci.id,1,100.0,None
3,2,SUBQUERY,website_visits,None,index,client_id,client_id,5,None,5000,100.0,Using index


### SHOW WARNINGS
#### содержит специальные маркеры, которые не являются допустимым SQL -выражением.

In [73]:
query = f'''
SHOW WARNINGS;
        '''
df = pd.read_sql_query(query, cnx)
for i in df['Message']:
    print(i)

/* select#1 */ select `otus_db`.`ci`.`first_name` AS `first_name`,`otus_db`.`ci`.`last_name` AS `last_name`,`otus_db`.`ci`.`email` AS `email`,count(distinct `otus_db`.`wv`.`id`) AS `total_visits`,count(distinct `otus_db`.`p`.`id`) AS `total_purchases`,sum(`otus_db`.`p`.`price`) AS `total_spent` from `otus_db`.`contact_info` `ci` left join `otus_db`.`website_visits` `wv` on((`otus_db`.`wv`.`client_id` = `otus_db`.`ci`.`id`)) left join `otus_db`.`purchases` `p` on((`otus_db`.`p`.`client_id` = `otus_db`.`ci`.`id`)) where <in_optimizer>(`otus_db`.`ci`.`id`,`otus_db`.`ci`.`id` in ( <materialize> (/* select#2 */ select `otus_db`.`website_visits`.`client_id` AS `cnt` from `otus_db`.`website_visits` group by `otus_db`.`website_visits`.`client_id` having (count(0) > 2) ), <primary_index_lookup>(`otus_db`.`ci`.`id` in <temporary table> on <auto_distinct_key> where ((`otus_db`.`ci`.`id` = `<materialized_subquery>`.`cnt`))))) group by `otus_db`.`ci`.`id` having (`total_spent` > 0) order by `total_

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


### EXPLAIN FORMAT = TREE 
#### позволяют читать план выполнения и информацию о стоимости запроса без использования SHOW WARNINGS.

In [76]:
query = f'''
EXPLAIN FORMAT = TREE 
SELECT
  ci.first_name,
  ci.last_name,
  ci.email,
  COUNT(DISTINCT wv.id) AS total_visits,
  COUNT(DISTINCT p.id) AS total_purchases,
  SUM(p.price) AS total_spent
FROM
  contact_info AS ci
  LEFT JOIN website_visits AS wv ON ci.id = wv.client_id
  LEFT JOIN purchases AS p ON ci.id = p.client_id
WHERE
  ci.id IN (
      SELECT client_id as cnt FROM website_visits group by client_id having count(*) > 2
    ) 
GROUP BY
  ci.id
  having total_spent > 0
ORDER BY
  total_spent DESC
  limit 10;

        '''
df = pd.read_sql_query(query, cnx)
for i in df['EXPLAIN']:
    print(i)

-> Limit: 10 row(s)
    -> Sort: total_spent DESC
        -> Filter: (total_spent > 0)
            -> Stream results  (cost=3637.12 rows=3000)
                -> Group aggregate: count(distinct wv.id), count(distinct p.id), sum(p.price)  (cost=3637.12 rows=3000)
                    -> Nested loop left join  (cost=3167.64 rows=4695)
                        -> Nested loop left join  (cost=1524.45 rows=4695)
                            -> Filter: <in_optimizer>(ci.id,ci.id in (select #2))  (cost=304.50 rows=3000)
                                -> Index scan on ci using PRIMARY  (cost=304.50 rows=3000)
                                -> Select #2 (subquery in condition; run only once)
                                    -> Filter: ((ci.id = `<materialized_subquery>`.cnt))  (cost=1322.85..1322.85 rows=1)
                                        -> Limit: 1 row(s)  (cost=1322.75..1322.75 rows=1)
                                            -> Index lookup on <materialized_subquery> using <aut

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


#### Еще более подробную информацию можно получить, заменив FORMAT = TREE на выражение ANALYZE, которое предоставляет MySQL с версии 8.0.18.

In [106]:
query = f'''
EXPLAIN ANALYZE 
SELECT
  ci.first_name,
  ci.last_name,
  ci.email,
  COUNT(DISTINCT wv.id) AS total_visits,
  COUNT(DISTINCT p.id) AS total_purchases,
  SUM(p.price) AS total_spent
FROM
  contact_info AS ci
  LEFT JOIN website_visits AS wv ON ci.id = wv.client_id
  LEFT JOIN purchases AS p ON ci.id = p.client_id
WHERE
  ci.id IN (
      SELECT client_id as cnt FROM website_visits group by client_id having count(*) > 2
    ) 
GROUP BY
  ci.id
  having total_spent > 0
ORDER BY
  total_spent DESC
  limit 10;

        '''
df = pd.read_sql_query(query, cnx)
for i in df['EXPLAIN']:
    print(i)

-> Limit: 10 row(s)  (actual time=42.513..42.514 rows=10 loops=1)
    -> Sort: total_spent DESC  (actual time=42.512..42.513 rows=10 loops=1)
        -> Filter: (total_spent > 0)  (actual time=5.161..41.919 rows=831 loops=1)
            -> Stream results  (cost=25842.42 rows=3000) (actual time=5.159..41.774 rows=869 loops=1)
                -> Group aggregate: count(distinct wv.id), count(distinct p.id), sum(p.price)  (cost=25842.42 rows=3000) (actual time=5.153..41.224 rows=869 loops=1)
                    -> Nested loop left join  (cost=21072.19 rows=47702) (actual time=5.070..37.236 rows=14821 loops=1)
                        -> Nested loop left join  (cost=4376.37 rows=15106) (actual time=4.963..15.678 rows=4790 loops=1)
                            -> Filter: <in_optimizer>(ci.id,ci.id in (select #2))  (cost=304.50 rows=3000) (actual time=4.942..8.970 rows=869 loops=1)
                                -> Index scan on ci using PRIMARY  (cost=304.50 rows=3000) (actual time=0.318..1.6

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


#### Оптимизация

#### Для таблиц contact_info, website_visits и purchases должны быть индексы на столбцы, используемые для соединения таблиц (ci.id, wv.client_id и p.client_id соответственно). Индексы помогают ускорить выполнение запроса, особенно при больших объемах данных.

#### Чем меньше значение rows  и чем больше значение filtered,- тем лучше. Однако, если значение rows слишком велико и filtered стремится к 100 %  - это очень плохо.

In [97]:
cur.execute('''
CREATE INDEX idx_cl_id_w
    ON website_visits (client_id);
            ''')

cur.execute('''
CREATE INDEX idx_cl_id_p
    ON purchases (client_id);
            ''')

In [98]:
query = f'''
EXPLAIN
SELECT ci.first_name,
       ci.last_name,
       ci.email,
       COUNT(DISTINCT wv.id) AS total_visits,
       COUNT(DISTINCT p.id) AS total_purchases,
       SUM(p.price) AS total_spent
FROM contact_info AS ci
LEFT JOIN website_visits AS wv ON ci.id = wv.client_id
LEFT JOIN purchases AS p ON ci.id = p.client_id
INNER JOIN (
  SELECT client_id
  FROM website_visits
  GROUP BY client_id
  HAVING COUNT(*) > 2
) AS wv2 ON ci.id = wv2.client_id
WHERE p.price > 0
GROUP BY ci.id
ORDER BY total_spent DESC
LIMIT 10;

        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,<derived2>,None,ALL,None,None,None,None,5000,100.00,Using where; Using temporary; Using filesort
1,1,PRIMARY,ci,None,eq_ref,PRIMARY,PRIMARY,4,wv2.client_id,1,100.00,None
2,1,PRIMARY,p,None,ref,idx_cl_id_p,idx_cl_id_p,5,wv2.client_id,3,33.33,Using where
3,1,PRIMARY,wv,None,ref,idx_cl_id_w,idx_cl_id_w,5,wv2.client_id,5,100.00,Using index
4,2,DERIVED,website_visits,None,index,idx_cl_id_w,idx_cl_id_w,5,None,5000,100.00,Using index


In [113]:
query = f'''
EXPLAIN ANALYZE 
SELECT
  ci.first_name,
  ci.last_name,
  ci.email,
  COUNT(DISTINCT wv.id) AS total_visits,
  COUNT(DISTINCT p.id) AS total_purchases,
  SUM(p.price) AS total_spent
FROM
  contact_info AS ci
  LEFT JOIN website_visits AS wv ON ci.id = wv.client_id
  LEFT JOIN purchases AS p ON ci.id = p.client_id
WHERE
  ci.id IN (
      SELECT client_id as cnt FROM website_visits group by client_id having count(*) > 2
    ) 
GROUP BY
  ci.id
  having total_spent > 0
ORDER BY
  total_spent DESC
  limit 10;

        '''
df = pd.read_sql_query(query, cnx)
for i in df['EXPLAIN']:
    print(i)

-> Limit: 10 row(s)  (actual time=39.276..39.278 rows=10 loops=1)
    -> Sort: total_spent DESC  (actual time=39.275..39.277 rows=10 loops=1)
        -> Filter: (total_spent > 0)  (actual time=8.708..38.903 rows=831 loops=1)
            -> Stream results  (cost=25842.42 rows=3000) (actual time=8.704..38.788 rows=869 loops=1)
                -> Group aggregate: count(distinct wv.id), count(distinct p.id), sum(p.price)  (cost=25842.42 rows=3000) (actual time=8.692..38.255 rows=869 loops=1)
                    -> Nested loop left join  (cost=21072.19 rows=47702) (actual time=8.538..34.963 rows=14821 loops=1)
                        -> Nested loop left join  (cost=4376.37 rows=15106) (actual time=8.511..17.851 rows=4790 loops=1)
                            -> Filter: <in_optimizer>(ci.id,ci.id in (select #2))  (cost=304.50 rows=3000) (actual time=8.481..12.508 rows=869 loops=1)
                                -> Index scan on ci using PRIMARY  (cost=304.50 rows=3000) (actual time=0.104..1.

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
